# Getting started with Task 4

Download the dataset from the [Dataset of Simulated Intracardiac Transmembrane Voltage Recordings and ECG Signals](https://library.ucsd.edu/dc/object/bb29449106) using the script `download_intracardiac_dataset.sh`:

```bash
source download_intracardiac_dataset.sh
```

## Load Modules and Preprocessing Functions

Load modules and preprocessing functions.

In [1]:
import glob, re, os
import numpy as np
import matplotlib.pyplot as plt
from typing import List
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf 


Load the `cardiac_ml_tools` module.

In [2]:
%run cardiac_ml_tools.py

## Load the dataset

In [3]:
data_dirs = []
regex = r'data_hearts_dd_0p2*'
DIR='../intracardiac_dataset/' # This should be the path to the intracardiac_dataset, it can be downloaded using data_science_challenge_2023/download_intracardiac_dataset.sh
for x in os.listdir(DIR):
    if re.match(regex, x):
        data_dirs.append(DIR + x)
file_pairs = read_data_dirs(data_dirs)
print('Number of file pairs: {}'.format(len(file_pairs)))
# example of file pair
print("Example of file pair:")
print("{}\n{}".format(file_pairs[0][0], file_pairs[0][1]))


Number of file pairs: 16117
Example of file pair:
../intracardiac_dataset/data_hearts_dd_0p2/pECGData_hearts_dd_0p2_volunteer.v10_pattern.0.npy
../intracardiac_dataset/data_hearts_dd_0p2/VmData_hearts_dd_0p2_volunteer.v10_pattern.0.npy


## Normalization

According to the article: "each V was normalized so that the value range was [0,1]." I'm interpreting 'value range' as the values in the "Getting transmembrane voltages" plot. I'm also assuming that for each case we'll normalize all of the 75 components at once. 

So we run through every case (16,117 total cases), subtract the minimum V value (to make smallest value = 0), then divide the resulting data by its maximum (to make range = [0,1]). Which means at least 75*500*16117*2 = 1.2e9 flops. Note this doesn't include the cost of loading each dataset nor the cost of loading ECG data and performing the standard_leads mapping. With operations of this scale it'll be important that we use any HPC resources we have. 


In [4]:
tic = time.time()
# Create an empty list to hold ECG and V matrices
ECG_list = [] 
V_list = []

# Loop over the cases and load the matrices
for case in range(len(file_pairs)):
    ## ECG 
    ECG = np.load(file_pairs[case][0])
    
    # Map from 10 leads to 12 
    ECG = get_standard_leads(ECG)
    
    # Normalize ECG so that the minimum and maximum for each col are separated by a distance of 1
    ECG = (ECG - ECG.min(axis=0))/(ECG.max(axis=0) - ECG.min(axis=0))

    # Append to ECG list 
    ECG_list.append(ECG)
    
    
    ## V
    V = np.load(file_pairs[case][1])

    # Normalize V so that "value range is [0,1]"
    V = V - np.min(V)
    V = V/np.max(V)

    # Append the normalized V matrix to the list 
    V_list.append(V)

# Reshape into tensor for dimension agreement
ECGtens = np.array(ECG_list)
Vtens = np.array(V_list)


toc = time.time()
print(f"time to normalize/load = {toc-tic}")

142.6536614894867


In [5]:
print(ECGtens[1,:,:].max(axis=0)-ECGtens[1,:,:].min(axis=0)) #proof that dist between max and min equals one 
print(ECGtens[1,:,:].shape) #Note that the ECG data is 500 by 12 -- not the other way around (as the paper describes)
print(Vtens[1,:,:].shape) #Similarly, 500 by 75, not 75 by 500

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
(500, 12)
(500, 75)


In [5]:
X, XX, y, yy = train_test_split(ECGtens, Vtens, test_size=0.05, random_state=42)


In [6]:
# Make the training set small for debugging
Xsmall, XXsmall, ysmall, yysmall = train_test_split(ECGtens[:4], Vtens[:4], 
                                                    test_size=0.5, random_state=42)

Below is the code for the squeezenet model as its described in the powerpoint slide. 

In [8]:
# class FireModule(tf.keras.layers.Layer):
#     def __init__(self, squeeze_channels, expand_channels):
#         super(FireModule, self).__init__()
#         self.squeeze = tf.keras.layers.Conv1D(squeeze_channels, 1, activation='relu')
#         self.expand1x1 = tf.keras.layers.Conv1D(expand_channels, 1, activation='relu')
#         self.expand3x3 = tf.keras.layers.Conv1D(expand_channels, 3, padding='same', activation='relu')

#     def call(self, x):
#         squeeze = self.squeeze(x)
#         expand1x1 = self.expand1x1(squeeze)
#         expand3x3 = self.expand3x3(squeeze)
#         return tf.concat([expand1x1, expand3x3], axis=0)

# # Define your model
# model = tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(500,12)),
#     tf.keras.layers.Conv1D(64, 3, strides=1, padding='same'),
#     tf.keras.layers.MaxPool1D(3, strides=1, padding='same'),
#     FireModule(16, 64),
#     FireModule(16, 64),
#     tf.keras.layers.MaxPool1D(3, strides=1, padding='same'),
#     FireModule(32, 128),
#     FireModule(32, 128),
#     tf.keras.layers.MaxPool1D(3, strides=1, padding='same'),
#     FireModule(48, 192),
#     FireModule(48, 192),
#     FireModule(64, 256),
#     FireModule(64, 256),
#     tf.keras.layers.Dropout(0.1),
#     tf.keras.layers.Conv1D(75, 1, padding='valid'),
#     tf.keras.layers.ReLU()
# ])

It doesn't actually run so I've commented it out. To make dimensions line up, I've added extra convolutional layers at the end. 

In [7]:
class FireModule(tf.keras.layers.Layer):
    def __init__(self, squeeze_channels, expand_channels):
        super(FireModule, self).__init__()
        self.squeeze = tf.keras.layers.Conv1D(squeeze_channels, 1, activation='relu')
        self.expand1x1 = tf.keras.layers.Conv1D(expand_channels, 1, activation='relu')
        self.expand3x3 = tf.keras.layers.Conv1D(expand_channels, 3, padding='same', activation='relu')

    def call(self, x):
        squeeze = self.squeeze(x)
        expand1x1 = self.expand1x1(squeeze)
        expand3x3 = self.expand3x3(squeeze)
        return tf.concat([expand1x1, expand3x3], axis=1)

# Define your model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(500,12)),

    tf.keras.layers.Conv1D(64, 3, strides=1, padding='same'),

    tf.keras.layers.MaxPool1D(3, strides=2, padding='same'),

    FireModule(16, 64),
    FireModule(16, 64), 

    tf.keras.layers.MaxPool1D(3, strides=2, padding='same'),

    FireModule(32, 128),
    FireModule(32, 128),

    tf.keras.layers.MaxPool1D(3, strides=2, padding='same'),

    FireModule(48, 192),
    FireModule(48, 192),
    FireModule(64, 256),
    FireModule(64, 256),

    tf.keras.layers.Dropout(0.1),

    tf.keras.layers.Conv1D(75, 1, strides=1),

    tf.keras.layers.AvgPool1D(),

    tf.keras.layers.Conv1D(75, 3, strides=2,padding='same'),

    tf.keras.layers.Conv1D(75, 3, strides=2,padding='same'),

    tf.keras.layers.Conv1D(75, 3, strides=2,padding='same'),

    tf.keras.layers.Conv1D(75, 3, strides=2,padding='same'),
    tf.keras.layers.ReLU()
])

In [8]:
model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.MeanSquaredError()])

One thing to notice is that my model isn't an exact replica of what they have in the paper. In the paper they mention that their model has 392,907 paramters while mine has 390,307:

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 500, 64)           2368      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 250, 64)          0         
 )                                                               
                                                                 
 fire_module (FireModule)    (None, 500, 64)           5264      
                                                                 
 fire_module_1 (FireModule)  (None, 1000, 64)          5264      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 500, 64)          0         
 1D)                                                             
                                                                 
 fire_module_2 (FireModule)  (None, 1000, 128)         1

To make sure there are no errors, let's run the model on a tiny subset of our total data. This should run pretty quick.

In [12]:
model.fit(Xsmall, ysmall, epochs=10, batch_size=32)

Epoch 1/10
1/1 [==============================] - 8s 8s/step - loss: 0.4165 - mean_squared_error: 0.4165
Epoch 2/10
1/1 [==============================] - 0s 198ms/step - loss: 0.4090 - mean_squared_error: 0.4090
Epoch 3/10
1/1 [==============================] - 0s 172ms/step - loss: 0.3930 - mean_squared_error: 0.3930
Epoch 4/10
1/1 [==============================] - 0s 151ms/step - loss: 0.3607 - mean_squared_error: 0.3607
Epoch 5/10
1/1 [==============================] - 0s 162ms/step - loss: 0.3026 - mean_squared_error: 0.3026
Epoch 6/10
1/1 [==============================] - 0s 155ms/step - loss: 0.2332 - mean_squared_error: 0.2332
Epoch 7/10
1/1 [==============================] - 0s 157ms/step - loss: 0.2853 - mean_squared_error: 0.2853
Epoch 8/10
1/1 [==============================] - 0s 192ms/step - loss: 0.1774 - mean_squared_error: 0.1774
Epoch 9/10
1/1 [==============================] - 0s 162ms/step - loss: 0.1603 - mean_squared_error: 0.1603
Epoch 10/10
1/1 [==============

Now for the real thing. 

In [ ]:
model.fit(X, y, epochs=30, batch_size=32)

Epoch 1/30
479/479 [==============================] - 1105s 2s/step - loss: 0.0444 - mean_squared_error: 0.0444
Epoch 2/30
479/479 [==============================] - 1105s 2s/step - loss: 0.0317 - mean_squared_error: 0.0317
Epoch 3/30
479/479 [==============================] - 1104s 2s/step - loss: 0.0290 - mean_squared_error: 0.0290
Epoch 4/30
479/479 [==============================] - 1090s 2s/step - loss: 0.0250 - mean_squared_error: 0.0250
Epoch 5/30
479/479 [==============================] - 1092s 2s/step - loss: 0.0218 - mean_squared_error: 0.0218
Epoch 6/30
479/479 [==============================] - 1075s 2s/step - loss: 0.0202 - mean_squared_error: 0.0202
Epoch 7/30
479/479 [==============================] - 1058s 2s/step - loss: 0.0194 - mean_squared_error: 0.0194
Epoch 8/30
479/479 [==============================] - 1066s 2s/step - loss: 0.0189 - mean_squared_error: 0.0189
Epoch 9/30
479/479 [==============================] - 1068s 2s/step - loss: 0.0185 - mean_squared_error:

Let's look at how the model did. First, let's create our predictions.

In [ ]:
ypred = model.predict(XX)
print(ypred.shape)

Compare the results for a random prediction.

In [1]:
num = np.random.randint(1,806)
true = yy[num,:,:]
pred = ypred[num,:,:]

# plot in row the tensors pECGData and ActTime with an arrow pointing to the activation time
row = 1
column = 3
plt.figure(figsize=(20, 5))
plt.subplot(row, column, 1)
# plot pECGData transposed
plt.imshow(true.T, cmap='jet', interpolation='nearest', aspect='auto')
plt.title('True data')
plt.subplot(row, column, 2)
# print a text "->"
plt.text(0.5, 0.5, 'Vs', fontsize=40, horizontalalignment='center', verticalalignment='center')
plt.axis('off')
plt.subplot(row, column, 3)
# plot Vm transposed
plt.imshow(pred.T, cmap='jet', interpolation='nearest', aspect='auto')
# not xticks
plt.xticks([])
plt.title('Reconstructed data')
plt.show()
plt.close()

NameError: name 'np' is not defined

Now let's consider the MSE between the true and predicted data. 

In [15]:
mse = model.evaluate(XX,yy)
print(mse)

26/26 [==============================] - 14s 506ms/step - loss: 0.0177 - mean_squared_error: 0.0177
[0.01770629920065403, 0.01770629733800888]


Finally, let's consider the average activation times... Which I think is what Mikel is doing...(?)

In [22]:
def activation_time_metric(true_data, predicted_data):
    case_metrics = []
    
    for true_case, predicted_case in zip(true_data, predicted_data):
        true_argmax = np.argmax(true_case, axis=0)
        predicted_argmax = np.argmax(predicted_case, axis=0)
        abs_diff = np.abs(true_argmax - predicted_argmax)
        case_metric = np.mean(abs_diff) #, np.var(abs_diff)
        case_metrics.append(case_metric)
    
    avg_metrics = np.mean(case_metrics, axis=0)
    var_metrics = np.var(case_metrics, axis=0)
    
    return avg_metrics, var_metrics

In [23]:
avg, var = activation_time_metric(yy,ypred)
print("Activation Time Average:", avg)
print("Activation Time Variance:", var)

Activation Time Average: 37.00251447477254
Activation Time Variance: 76.85077548055972


Which are far from Mikel's results... 

There's still work to be done on this model. If someone wants to pick up from where I left off, below I provide some code that helps with debugging/playing around: 

In [ ]:
# #############  # #FOR TESTING

# class FireModule(tf.keras.layers.Layer):
#     def __init__(self, squeeze_channels, expand_channels):
#         super(FireModule, self).__init__()
#         self.squeeze = tf.keras.layers.Conv1D(squeeze_channels, 1, activation='relu')
#         self.expand1x1 = tf.keras.layers.Conv1D(expand_channels, 1, activation='relu')
#         self.expand3x3 = tf.keras.layers.Conv1D(expand_channels, 3, padding='same', activation='relu')

#     def call(self, x):
#         # tf.print('Input shape:', x.shape)
#         squeeze = self.squeeze(x)
#         expand1x1 = self.expand1x1(squeeze)
#         expand3x3 = self.expand3x3(squeeze)
#         # tf.print('Output shape:', x.shape)
#         return tf.concat([expand1x1, expand3x3], axis=1)
    
# class PrintMod(tf.keras.layers.Layer):
#     def __init__(self, message=''):
#         super(PrintMod, self).__init__()
#         self.message = message

#     def call(self, x):
#         tf.print(self.message, 'shape:', x.shape)
#         return x

# # # Define your model
# model = tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(500,12)),
#     # tf.keras.layers.Input(shape=(12,500)),
#     PrintMod('After input'),

#     tf.keras.layers.Conv1D(64, 3, strides=1, padding='same'),
#     # tf.keras.layers.Conv1D(64, 3, strides=2, padding='same'), #output is 12 by 250
#     PrintMod('After Conv1D(64, 3, strides=2, padding=same)'),

#     # tf.keras.layers.MaxPool1D(3, strides=1, padding='same'),
#     tf.keras.layers.MaxPool1D(3, strides=2, padding='same'), #output is 12 by 125
#     PrintMod('After MaxPool1D(3, strides=2, padding=same)'),

#     FireModule(16, 64), #output is 64 by 250 (I think)
#     PrintMod('After Fire 16, 64'),
#     FireModule(16, 64), 
#     PrintMod('After Fire 16, 64'),

#     # tf.keras.layers.MaxPool1D(3, strides=1, padding='same'),
#     tf.keras.layers.MaxPool1D(3, strides=2, padding='same'),
#     PrintMod('After MaxPool1D(3, strides=2, padding=same)'),

#     FireModule(32, 128),
#     PrintMod('After fire 32, 128'),
#     FireModule(32, 128),
#     PrintMod('After fire 32, 128'),


#     # tf.keras.layers.MaxPool1D(3, strides=1, padding='same'),
#     tf.keras.layers.MaxPool1D(3, strides=2, padding='same'),
#     PrintMod('After MaxPool1D(3, strides=2, padding=same)'),

#     FireModule(48, 192),
#     PrintMod('After fire 48, 192'),
#     FireModule(48, 192),
#     PrintMod('After fire 48, 192'),
#     FireModule(64, 256),
#     PrintMod('After fire 64, 256'),
#     FireModule(64, 256),
#     PrintMod('After fire 64, 256'),

#     tf.keras.layers.Dropout(0.1),
#     # tf.keras.layers.Conv1D(75, 1),
#     tf.keras.layers.Conv1D(75, 1, strides=1),
#     PrintMod('After final layer'),
#     # # tf.keras.layers.ReLU()

#     tf.keras.layers.AvgPool1D(),
#     PrintMod('avgpool1d'),

#     tf.keras.layers.Conv1D(75, 3, strides=2,padding='same'),
#     # tf.keras.layers.Conv1D(64, 3, strides=2, padding='same'), #output is 12 by 250
#     PrintMod('After Conv1D'),

#     tf.keras.layers.Conv1D(75, 3, strides=2,padding='same'),
#     # tf.keras.layers.Conv1D(64, 3, strides=2, padding='same'), #output is 12 by 250
#     PrintMod('After Conv1D'),

#     tf.keras.layers.Conv1D(75, 3, strides=2,padding='same'),
#     # tf.keras.layers.Conv1D(64, 3, strides=2, padding='same'), #output is 12 by 250
#     PrintMod('After Conv1D'),

#     tf.keras.layers.Conv1D(75, 3, strides=2,padding='same'),
#     # tf.keras.layers.Conv1D(64, 3, strides=2, padding='same'), #output is 12 by 250
#     tf.keras.layers.ReLU(),
#     PrintMod('After Conv1D')

    
# ])